EUSS R1 Debug For Slab Warmup Issue

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import plotly.express as px
import getpass

user = getpass.getuser()
print(f"The user is {user}")

The user is lliu2


In [2]:
# set path
if user == "ylou2":
    euss_dir = 'data_/euss_res_final_2018_550k_20220901/' # TODO
elif user == "lliu2":
    euss_dir = "/Volumes/Lixi_Liu/euss_aws"
else:
    print(f"new user={user}, set euss_dir path here with an elif statement")

euss_dir = Path(euss_dir).resolve()
output_dir = Path(".").resolve()
output_dir.mkdir(exist_ok=True)

print(f"euss_dir: {euss_dir}")
print(f"output_dir: {output_dir}")

euss_dir: /Volumes/Lixi_Liu/euss_aws
output_dir: /Users/lliu2/Documents/GitHub/resstock/euss_cleap/euss_debug


In [3]:
dfb = pd.read_parquet(euss_dir / "results_up00.parquet")
dfb = dfb.loc[dfb["completed_status"] == "Success"].set_index("building_id")
dfb

,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,build_existing_model.ashrae_iecc_climate_zone_2004,...,qoi_report.qoi_average_minimum_daily_use_heating_kw,qoi_report.qoi_average_minimum_daily_use_overlap_kw,qoi_report.qoi_average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.qoi_average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.qoi_average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.qoi_average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.qoi_peak_magnitude_timing_hour,qoi_report.qoi_peak_magnitude_use_kw,build_existing_model.area_median_income,rep_income
building_id,,,,,,,,,,,,,,,,,,,,,
1,211,2022-09-02 02:24:15,2022-09-02 02:24:53,Success,None,None,,Non-CBSA South Atlantic,True,4A,...,0.32,0.30,15.09,17.91,5.42,4.50,5086.0,7.21,80-100%,63538.120500
2,591,2022-09-02 08:03:09,2022-09-02 08:03:43,Success,None,None,,Non-CBSA Mountain,True,6B,...,0.50,0.31,12.73,15.91,3.37,5.50,8494.0,6.73,150%+,110812.513064
4,344,2022-09-02 01:48:45,2022-09-02 01:49:51,Success,None,None,,Non-CBSA West South Central,True,2A,...,0.27,0.07,15.09,8.27,6.26,10.30,391.0,14.19,0-30%,10545.690081
5,57,2022-09-02 02:07:07,2022-09-02 02:07:41,Success,None,None,,Non-CBSA West North Central,True,4A,...,2.58,0.57,14.18,9.09,5.81,15.34,391.0,18.66,150%+,108847.716464
6,218,2022-09-02 00:24:08,2022-09-02 00:24:32,Success,None,None,,"CBSA Miami-Fort Lauderdale-West Palm Beach, FL",True,2A,...,0.57,0.57,18.00,17.00,5.15,3.07,7890.0,6.17,30-60%,37476.379500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549996,410,2022-09-02 01:36:12,2022-09-02 01:36:38,Success,None,None,,"CBSA Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",True,4A,...,1.47,0.19,17.36,7.55,1.21,5.39,125.0,6.58,0-30%,5402.820544
549997,32,2022-09-01 22:56:23,2022-09-01 22:56:52,Success,None,None,,"CBSA New York-Newark-Jersey City, NY-NJ-PA",True,5A,...,0.46,0.47,17.45,18.55,1.59,1.24,5923.0,1.70,120-150%,148804.550862
549998,23,2022-09-02 02:07:05,2022-09-02 02:07:43,Success,None,None,,"CBSA Los Angeles-Long Beach-Anaheim, CA",True,3B,...,0.54,0.53,13.64,16.82,8.83,4.44,2365.0,9.37,0-30%,23770.007360


In [ ]:
heating_cols = [x for x in dfb.columns if ".end_use" in x and ("_heating_m_btu" in x or "_heating_heat_pump_backup_m_btu" in x)]
print(f"Calculate total heating by combining: {heating_cols}")

dfb["total_heating_m_btu"] = dfb[heating_cols].sum(axis=1)

### 1. Compare delivered heating and unmet heating hours between baseline and upgrade

In [ ]:
upg = 4
dfu = pd.read_parquet(euss_dir / f"results_up{upg:02d}.parquet")
dfu = dfu.loc[dfu["completed_status"] == "Success"].set_index("building_id")
upgrade_name = dfu["apply_upgrade.upgrade_name"].unique()[0]

print(f"Calculate total heating")
dfu["total_heating_m_btu"] = dfu[heating_cols].sum(axis=1)

dfu

In [ ]:
### Apply filters - remove none heating fuel
cond = dfb["build_existing_model.heating_fuel"] != "None"

dfbf = dfb.loc[cond]
dfuf = dfu.loc[sorted(set(dfbf.index).intersection(set(dfu.index)))]

In [ ]:
cond = dfbf["report_simulation_output.load_heating_delivered_m_btu"]==0
dfbf.loc[cond, [
    "build_existing_model.state", "build_existing_model.heating_fuel", "build_existing_model.hvac_heating_efficiency", "build_existing_model.heating_setpoint",
    "report_simulation_output.load_heating_delivered_m_btu", "total_heating_m_btu"]]
          #"report_simulation_output.end_use_electricity_heating_m_btu"]]

In [ ]:
cond = dfbf["report_simulation_output.load_heating_delivered_m_btu"]==0
gb = [
    # "build_existing_model.heating_fuel", 
    "build_existing_model.ashrae_iecc_climate_zone_2004"
    ]

print("% Homes with 0 delivered heating:")
dfbf.loc[cond].groupby(gb)["build_existing_model.sample_weight"].count() / dfbf.groupby(gb)["build_existing_model.sample_weight"].count() * 100

In [ ]:
# Filter to those with at least 1 mmbtu of delivered heating
n_bfr = len(dfbf)
cond = dfbf["report_simulation_output.load_heating_delivered_m_btu"]>= 1
dfbf = dfbf.loc[cond]
dfuf = dfuf.loc[sorted(set(dfbf.index).intersection(set(dfuf.index)))]
n_aft = len(dfbf)

print(f"Filtering to those with ≥1 mmbtu reduces baseline stock from {n_bfr} to {n_aft} ({(n_aft-n_bfr)/n_bfr*100:.02f}%)")

In [ ]:
metric = "report_simulation_output.load_heating_delivered_m_btu"
deliv_heat_diff = (dfuf[metric] - dfbf[metric]) / dfbf[metric] * 100 # pct
deliv_heat_diff = deliv_heat_diff.rename("pct_diff_in_delivered_heating")

deliv_heat_diff.describe()

In [ ]:
metric = "report_simulation_output.unmet_hours_heating_hr"
unmet_heat_diff = (dfuf[metric] - dfbf[metric]) # hours
unmet_heat_diff = unmet_heat_diff.rename("diff_in_unmet_heating_hours"
)

unmet_heat_diff.describe()

In [ ]:
dfp = pd.concat([dfbf[[
    "build_existing_model.heating_fuel",
    "build_existing_model.hvac_heating_type"]], 
                 unmet_heat_diff, deliv_heat_diff], axis=1)
dfp

In [ ]:
# within x hours of unmet heating diff, how many over y% discrepancy in delivered heating?
threshold_diff_in_unmet_heating_hours = 10 # <--- inclusive, within this condition
threshold_pct_diff_in_delivered_heating = 1 # <--- exclusive, beyond this condition

cond = ((dfp["diff_in_unmet_heating_hours"]<=threshold_diff_in_unmet_heating_hours) & (dfp["diff_in_unmet_heating_hours"]>=-threshold_diff_in_unmet_heating_hours))
cond &= ((dfp["pct_diff_in_delivered_heating"]>threshold_pct_diff_in_delivered_heating) | (dfp["pct_diff_in_delivered_heating"]<-threshold_pct_diff_in_delivered_heating))

print(f"Within {threshold_diff_in_unmet_heating_hours} hours of unmet heating diff, "
      f"{cond.sum() / len(dfp)*100:.02f}% of stock have discrepancy over {threshold_pct_diff_in_delivered_heating}% in delivered heating")


In [ ]:
cond2 = dfp["build_existing_model.hvac_heating_type"].isin(['Non-Ducted Heating', 'Non-Ducted Heat Pump'])
cond3 = cond & cond2
print(f"Within {threshold_diff_in_unmet_heating_hours} hours of unmet heating diff, "
      f"{cond3.sum() / len(dfp.loc[cond2])*100:.02f}% of stock with ductless heating have discrepancy over {threshold_pct_diff_in_delivered_heating}% in delivered heating")

In [ ]:
cond2 = dfp["build_existing_model.hvac_heating_type"].isin(['Ducted Heating', 'Ducted Heat Pump'])
cond3 = cond & cond2
print(f"Within {threshold_diff_in_unmet_heating_hours} hours of unmet heating diff, "
      f"{cond3.sum() / len(dfp.loc[cond2])*100:.02f}% of stock with ducted heating have discrepancy over {threshold_pct_diff_in_delivered_heating}% in delivered heating")

In [ ]:
px.scatter(dfp, x="diff_in_unmet_heating_hours", y="pct_diff_in_delivered_heating",
           color="build_existing_model.hvac_heating_type")

### 2. Check first timestep living space temp vs. heating setpoint

Athena Query:

WITH bl AS (
SELECT building_id, CAST(RTRIM("build_existing_model.heating_setpoint", 'F') AS INT) AS heating_setpoint_f
FROM "euss-final"."euss_res_final_2018_550k_20220901_baseline"
),

ts AS (
SELECT upgrade, building_id, time, "zone_mean_air_temperature__living_space__c"*9/5+32 AS living_space_temp_f
FROM "euss-final"."euss_res_final_2018_550k_20220901_timeseries"
WHERE time = timestamp '2018-01-01 00:15:00.000'
)

SELECT upgrade, ts.building_id, time, living_space_temp_f, heating_setpoint_f, (living_space_temp_f - heating_setpoint_f) AS temp_delta_f 
FROM ts LEFT JOIN bl ON ts.building_id = bl.building_id
ORDER BY 1, 2


In [16]:
df = pd.read_csv(euss_dir / "EUSS_r1_starting_timestep_temp.csv") # file queried on AWS
df["upgrade"] = pd.Categorical(df["upgrade"].astype(str), categories=[f"{x}" for x in range(11)])
df = df.sort_values(by=["upgrade", "building_id"])
df

,upgrade,building_id,time,heating_setpoint_f,living_space_temp_f,temp_delta_f
0,0,1,2018-01-01 00:15:00.000,68.0,68.000000,0.000000e+00
1,0,2,2018-01-01 00:15:00.000,70.0,67.000000,-3.000000e+00
2,0,4,2018-01-01 00:15:00.000,55.0,55.000000,4.263256e-14
3,0,5,2018-01-01 00:15:00.000,67.0,67.000000,-8.526513e-14
4,0,6,2018-01-01 00:15:00.000,68.0,74.709755,6.709755e+00
...,...,...,...,...,...,...
1629663,10,549996,2018-01-01 00:15:00.000,68.0,68.000107,1.073134e-04
1629664,10,549997,2018-01-01 00:15:00.000,72.0,70.000011,-1.999989e+00
1629665,10,549998,2018-01-01 00:15:00.000,60.0,74.300035,1.430004e+01
1629666,10,549999,2018-01-01 00:15:00.000,60.0,55.284158,-4.715842e+00


In [17]:
# add metadata
df = df.join(dfb[[
    "build_existing_model.heating_fuel",
    "build_existing_model.hvac_heating_type",
    "build_existing_model.geometry_foundation_type"
]], on="building_id").rename(columns=lambda x: x.removeprefix("build_existing_model."))

df

,upgrade,building_id,time,heating_setpoint_f,living_space_temp_f,temp_delta_f,heating_fuel,hvac_heating_type,geometry_foundation_type
0,0,1,2018-01-01 00:15:00.000,68.0,68.000000,0.000000e+00,Propane,Ducted Heating,Slab
1,0,2,2018-01-01 00:15:00.000,70.0,67.000000,-3.000000e+00,Natural Gas,Ducted Heating,Heated Basement
2,0,4,2018-01-01 00:15:00.000,55.0,55.000000,4.263256e-14,Electricity,Ducted Heating,Ambient
3,0,5,2018-01-01 00:15:00.000,67.0,67.000000,-8.526513e-14,Electricity,Ducted Heating,Slab
4,0,6,2018-01-01 00:15:00.000,68.0,74.709755,6.709755e+00,None,None,Vented Crawlspace
...,...,...,...,...,...,...,...,...,...
1629663,10,549996,2018-01-01 00:15:00.000,68.0,68.000107,1.073134e-04,Electricity,Non-Ducted Heating,Vented Crawlspace
1629664,10,549997,2018-01-01 00:15:00.000,72.0,70.000011,-1.999989e+00,Natural Gas,Ducted Heating,Unheated Basement
1629665,10,549998,2018-01-01 00:15:00.000,60.0,74.300035,1.430004e+01,Electricity,Non-Ducted Heating,Slab
1629666,10,549999,2018-01-01 00:15:00.000,60.0,55.284158,-4.715842e+00,Propane,Ducted Heating,Vented Crawlspace


In [18]:
def bin_temp_delta(x):
    if x<-5:
        return "<-5 degF"
    if (x>=-5) & (x<-3):
        return "[-5, -3) degF"
    if (x>=-3) & (x<-1):
        return "[-3, -1) degF"
    if (x>=-1) & (x<1):
        return "[-1, 1) degF"
    if (x>=1) & (x<3):
        return "[1, 3) degF"
    if (x>=3) & (x<5):
        return "[3, 5) degF"
    if x>=5:
        return "5+ degF"
    
df["temp_delta_bin"] = df["temp_delta_f"].apply(lambda x: bin_temp_delta(x))
df

,upgrade,building_id,time,heating_setpoint_f,living_space_temp_f,temp_delta_f,heating_fuel,hvac_heating_type,geometry_foundation_type,temp_delta_bin
0,0,1,2018-01-01 00:15:00.000,68.0,68.000000,0.000000e+00,Propane,Ducted Heating,Slab,"[-1, 1) degF"
1,0,2,2018-01-01 00:15:00.000,70.0,67.000000,-3.000000e+00,Natural Gas,Ducted Heating,Heated Basement,"[-5, -3) degF"
2,0,4,2018-01-01 00:15:00.000,55.0,55.000000,4.263256e-14,Electricity,Ducted Heating,Ambient,"[-1, 1) degF"
3,0,5,2018-01-01 00:15:00.000,67.0,67.000000,-8.526513e-14,Electricity,Ducted Heating,Slab,"[-1, 1) degF"
4,0,6,2018-01-01 00:15:00.000,68.0,74.709755,6.709755e+00,None,None,Vented Crawlspace,5+ degF
...,...,...,...,...,...,...,...,...,...,...
1629663,10,549996,2018-01-01 00:15:00.000,68.0,68.000107,1.073134e-04,Electricity,Non-Ducted Heating,Vented Crawlspace,"[-1, 1) degF"
1629664,10,549997,2018-01-01 00:15:00.000,72.0,70.000011,-1.999989e+00,Natural Gas,Ducted Heating,Unheated Basement,"[-3, -1) degF"
1629665,10,549998,2018-01-01 00:15:00.000,60.0,74.300035,1.430004e+01,Electricity,Non-Ducted Heating,Slab,5+ degF
1629666,10,549999,2018-01-01 00:15:00.000,60.0,55.284158,-4.715842e+00,Propane,Ducted Heating,Vented Crawlspace,"[-5, -3) degF"


In [28]:
for hc in ["heating_fuel", "hvac_heating_type", "geometry_foundation_type"]:
    df_temp = (df.groupby([
        "upgrade", 
        hc, 
        "temp_delta_bin"])["building_id"].count() / df.groupby([
        "upgrade", 
        hc])["building_id"].count()).rename("fraction").reset_index()

    df_temp["temp_delta_bin"] = pd.Categorical(df_temp["temp_delta_bin"], categories = [
        "<-5 degF",
        "[-5, -3) degF",
        "[-3, -1) degF",
        "[-1, 1) degF",
        "[1, 3) degF",
        "[3, 5) degF",
        "5+ degF",
    ], ordered=True)

    df_temp["upgrade"] = pd.Categorical(df_temp["upgrade"].astype(str), categories=[f"{x}" for x in range(11)])
    df_temp = df_temp.sort_values(by=["upgrade", hc, "temp_delta_bin"])
    
    print(df_temp)
    
    px.bar(df_temp, 
       x="temp_delta_bin", y="fraction", color="temp_delta_bin",
             facet_col=hc,
             facet_row="upgrade",
       height=1200, width=200*df_temp[hc].nunique()
            ).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])).write_image(
    output_dir / f"temp_diff_by_{hc}.png")


    upgrade heating_fuel temp_delta_bin  fraction
1         0  Electricity       <-5 degF  0.124698
4         0  Electricity  [-5, -3) degF  0.079901
3         0  Electricity  [-3, -1) degF  0.137885
2         0  Electricity   [-1, 1) degF  0.517548
5         0  Electricity    [1, 3) degF  0.032208
..      ...          ...            ...       ...
458      10      Propane  [-3, -1) degF  0.191214
457      10      Propane   [-1, 1) degF  0.545483
460      10      Propane    [1, 3) degF  0.019743
461      10      Propane    [3, 5) degF  0.013844
455      10      Propane        5+ degF  0.031659

[462 rows x 4 columns]
    upgrade hvac_heating_type temp_delta_bin  fraction
1         0  Ducted Heat Pump       <-5 degF  0.119758
4         0  Ducted Heat Pump  [-5, -3) degF  0.078685
3         0  Ducted Heat Pump  [-3, -1) degF  0.164274
2         0  Ducted Heat Pump   [-1, 1) degF  0.498797
5         0  Ducted Heat Pump    [1, 3) degF  0.031984
..      ...               ...            ...  